In [43]:
# Packages
import pandas as pd
import os
import datetime as dt
import numpy as np
import pyproj

geod = pyproj.Geod(ellps='WGS84')
from multiprocessing.pool import Pool



In [44]:
# Settings (file names and directories)

LABELS_FILE = 'labels.txt'
MAIN_FOLDER = './data/'
TRAJ_FOLDER = 'Trajectory/'
OUTPUT_FOLDER = './data_preprocessed/'
POOLSIZE = 4



In [45]:
# Distance between 2 points
def calculate_distance(long1, lat1, long2, lat2):
    if lat1 == lat2 and long1 == long2:
        return 0
    if False in np.isfinite([long1, long2, lat1, lat2]):
        return np.nan
    if lat1 < -90 or lat1 > 90 or lat2 < -90 or lat2 > 90:
        #raise ValueError('The range of latitudes seems to be invalid.')
        return np.nan
    if long1 < -180 or long1 > 180 or long2 < -180 or long2 > 180:
        return np.nan
        #raise ValueError('The ranя ge of longitudes seems to be invalid.')
    angle1,angle2,distance = geod.inv(long1, lat1, long2, lat2)
    return distance

# Get velocity distance/timedelta
def calculate_velocity(distance, timedelta):
    if timedelta.total_seconds() == 0: return np.nan
    return distance / timedelta.total_seconds()

# Get acceleration diff(velocity)/timedelta
def calculate_acceleration(velocity, velocity_next_position, timedelta):
    delta_v = velocity_next_position - velocity
    if timedelta.total_seconds() == 0: return np.nan
    return delta_v / timedelta.total_seconds()

In [46]:
headers_trajectory = ['lat', 'long', 'null', 'altitude','timestamp_float', 'date', 'time']

# Convert str datetime to datetime format
def to_datetime(string):
    return dt.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')

# Load labels in the folder
def load_labels_df(filename):
    df = pd.read_csv(filename, sep='\t')
    df['start_time'] = df['Start Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['end_time'] = df['End Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['labels'] = df['Transportation Mode']
    df = df.drop(['End Time', 'Start Time', 'Transportation Mode'], axis=1)
    return df

# Load trajectory
def load_trajectory_df(full_filename):
    subfolder = full_filename.split('/')[-3]
    trajectory_id = full_filename.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(full_filename, skiprows = 6, header = None, names = headers_trajectory)
   
    df['datetime'] = df.apply(lambda z: to_datetime(z.date + ' ' + z.time), axis=1)
    df['datetime_next_position'] = df['datetime'].shift(-1)
    df['timedelta'] = df.apply(lambda z: z.datetime_next_position - z.datetime, axis=1)
    df = df.drop(['datetime_next_position'], axis=1)
    df = df.drop(['null', 'timestamp_float', 'date', 'time'], axis=1)
    
    df['long_next_position'] = df['long'].shift(-1)
    df['lat_next_position'] = df['lat'].shift(-1)
    df['distance'] = df.apply(lambda z: calculate_distance(z.long, z.lat, z.long_next_position, z.lat_next_position), axis=1)
    df = df.drop(['long_next_position', 'lat_next_position'], axis=1)
    
    df['velocity'] = df.apply(lambda z: calculate_velocity(z.distance, z.timedelta), axis=1)
    df['velocity_next_position'] = df['velocity'].shift(-1)
    df['acceleration'] = df.apply(lambda z: calculate_acceleration(z.velocity, z.velocity_next_position, z.timedelta), axis=1)
    df = df.drop(['velocity_next_position'], axis=1)
    
    df['trajectory_id'] = trajectory_id
    df['subfolder'] = subfolder
    df['labels'] = ''
    calculate_agg_features(df)
    return df

#This method calculates the aggregated feature and 
#saves them in the original df
def calculate_agg_features(df):
    v_ave = np.nanmean(df['velocity'].values)
    v_med = np.nanmedian(df['velocity'].values)
    v_max = np.nanmax(df['velocity'].values)
    v_std = np.nanstd(df['velocity'].values)
    
    a_ave = np.nanmean(df['acceleration'].values)
    a_med = np.nanmedian(df['acceleration'].values)
    a_max = np.nanmax(df['acceleration'].values)
    a_std = np.nanstd(df['acceleration'].values)
   
    df.loc[:, 'v_ave'] = v_ave
    df.loc[:, 'v_med'] = v_med
    df.loc[:, 'v_max'] = v_max
    df.loc[:, 'v_std'] = v_std
    
    df.loc[:, 'a_ave'] = a_ave
    df.loc[:, 'a_med'] = a_med
    df.loc[:, 'a_max'] = a_max
    df.loc[:, 'a_std'] = a_std

In [48]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
directories = os.listdir(MAIN_FOLDER)

# Garbage collector used to collect garbage 
# becouse OS handles it slow
import gc

# Go to the dirs and try load it.
# Save in the preprocessed folder
for subfolder in directories:
    list_df_traj = []
    subfolder_ = MAIN_FOLDER + subfolder + '/'
    traj_folder = MAIN_FOLDER + subfolder + '/' + TRAJ_FOLDER
    traj_files = os.listdir(traj_folder)
    
    traj_files_full_path = [traj_folder + traj_file for traj_file in traj_files]
    print(subfolder, len(traj_files_full_path))
    
    #multiprocessing does not work well in the jupyter notebook environment.
    #outside of jupyter you can use multiprocessing to speed up the process
#     pool = Pool(4)
#     for df in pool.imap_unordered(load_trajectory_df, traj_files_full_path):
#        list_df_traj.append(df)
    
    for i, file in enumerate(traj_files_full_path):
        print(subfolder + ' -> index:' + str(i))
        list_df_traj.append(load_trajectory_df(file))
    
    df_traj_all = pd.concat(list_df_traj)
    list_df_traj = []
    
    if LABELS_FILE in os.listdir(subfolder_):
        filename = subfolder_ + LABELS_FILE
        df_labels = load_labels_df(filename)
        for idx in df_labels.index.values:
            st = df_labels.ix[idx]['start_time']
            et = df_labels.ix[idx]['end_time']
            labels = df_labels.ix[idx]['labels']
            if labels:
                df_traj_all.loc[(df_traj_all['datetime'] >= st) & 
                                (df_traj_all['datetime'] <= et), 'labels'] = labels

    output_filename = OUTPUT_FOLDER + subfolder + '.csv'
    df_traj_all.to_csv(output_filename)
    del df_traj_all
    gc.collect()


142 156
055 24
106 3
024 101


/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


145 6
032 16
062 706
038 110
018 50
084 215
009 49
153 2024
044 72
165 26
092 157
093 23
101 68
060 2
013 144
163 809
122 16
022 146
045 9
005 86
114 23
052 104


Process ForkPoolWorker-212:
Process ForkPoolWorker-119:
Process ForkPoolWorker-66:
Process ForkPoolWorker-198:
Process ForkPoolWorker-237:
Process ForkPoolWorker-88:
Process ForkPoolWorker-245:
Process ForkPoolWorker-230:
Process ForkPoolWorker-43:
Process ForkPoolWorker-164:
Process ForkPoolWorker-182:
Process ForkPoolWorker-236:
Process ForkPoolWorker-101:
Process ForkPoolWorker-46:
Process ForkPoolWorker-118:
Process ForkPoolWorker-91:
Process ForkPoolWorker-53:
Process ForkPoolWorker-126:
Process ForkPoolWorker-79:
Process ForkPoolWorker-100:
Process ForkPoolWorker-140:
Process ForkPoolWorker-137:
Process ForkPoolWorker-186:
Process ForkPoolWorker-106:
Process ForkPoolWorker-207:
Process ForkPoolWorker-195:
Process ForkPoolWorker-219:
Process ForkPoolWorker-193:
Process ForkPoolWorker-191:
Process ForkPoolWorker-70:
Process ForkPoolWorker-208:
Process ForkPoolWorker-84:
Process ForkPoolWorker-142:
Process ForkPoolWorker-253:
Process ForkPoolWorker-115:
Process ForkPoolWorker-98:
Pr

KeyboardInterrupt: 

...

  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/frame.py", line 4972, in _apply_standard
    for i, v in enumerate(series_gen):
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml

KeyboardInterrupt
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
 

  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "<ipython-input-46-396e74ce57f5>", line 21, in

  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/series.py", line 623, in __getitem__
    result = self.index.get_value(self, key)
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/internals.py", line 4411, in _block
    return self.blocks[0]
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    tas

  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "pandas/_libs/src/reduce.pyx", line 134, in pandas._libs.lib.Reducer.get_result
KeyboardInterrupt
  File "/home/xyz/Program/miniconda3/envs/ml/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._sem

Result

1) All GPS and labels data converted to the one CSV file.